<a href="https://colab.research.google.com/github/AAhmadS/deeplearning-HW4/blob/main/NLP_Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/deep learning

/content/drive/MyDrive/deep learning


In [23]:
!pip install torchtext==0.5.0
!git clone https://github.com/amnghd/Persian_poems_corpus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'Persian_poems_corpus' already exists and is not an empty directory.


In [21]:
!pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 69.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 KB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 29.2 MB/s eta 0:00:00


In [24]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import hazm
from cleantext import clean

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os

import copy
import collections

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


#Section A :
##loading data

In [204]:
poets = ['khajoo','asad','bahar','feyz','asadi','hafez','jami','kamal','moulavi','parvin']
poems_data = []
label = []

for i,p in enumerate(poets):
  data = open('Persian_poems_corpus/normalized/'+p+'_norm.txt').read()
  poems = data.lower().split("\n")[1:]
  for poem in poems:
    if'  \t' != poem :
      poems_data.append(poem)
      label.append(i)
poems = pd.DataFrame({"poem": poems_data,
                     "poet": label})
poems.to_csv('poems.csv')

In [205]:
poems.head(5)

,poem,poet
0,ای غره ماه از اثر صنع تو غرا,0
1,وی طره شب از دم لطف تو مطرا,0
2,نوک قلم صنع تودر مبدا فطرت,0
3,انگیخته برصفحه کن صورت اشیا,0
4,سجاده نشینان نه ایوان فلک را,0


In [206]:
poems['poem_len_by_words'] = poems['poem'].apply(lambda t: len(hazm.word_tokenize(t)))

min_max_len = poems["poem_len_by_words"].min(), poems["poem_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 0 	Max: 15


In [207]:
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='poem_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [208]:
data_gl_than(poems, 15, 3)

Texts with word length of greater than 3 and less than 15 includes 86.67% of the whole!


In [209]:
poems['poem_len_by_words'] = poems['poem_len_by_words'].apply(lambda len_t: len_t if 3 < len_t <= 15 else None)
poems = poems.dropna(subset=['poem_len_by_words'])
poems = poems.reset_index(drop=True)

In [210]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=poems['poem_len_by_words']
))

fig.update_layout(
    title_text='Distribution of word counts within poems',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [211]:
fig = go.Figure()

groupby_rate = poems.groupby('poet')['poet'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_rate.index)),
    y=groupby_rate.tolist(),
    text=groupby_rate.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of poets within poems',
    xaxis_title_text='poet',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

##Balancing

In [212]:
data0 = poems[poems['poet'] == 0]
data1 = poems[poems['poet'] == 1]
data2 = poems[poems['poet'] == 2]
data3 = poems[poems['poet'] == 3]
data4 = poems[poems['poet'] == 4]
data5 = poems[poems['poet'] == 5]
data6 = poems[poems['poet'] == 6]
data7 = poems[poems['poet'] == 7]
data8 = poems[poems['poet'] == 8]
data9 = poems[poems['poet'] == 9]

cutting_point = 9430

if cutting_point <= len(data0):
    data0 = data0.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data1):
    data1 = data1.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data2):
    data2 = data2.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data3):
    data3 = data3.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data4):
    data4 = data4.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data5):
    data5 = data5.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data6):
    data6 = data6.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data7):
    data7 = data7.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data8):
    data8 = data8.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(data9):
    data9 = data9.sample(n=cutting_point).reset_index(drop=True)

new_data = pd.concat([data0,data1,data2,data3,data4,data5,data6,data7,data8,data9])
new_data = new_data.sample(frac=1).reset_index(drop=True)

In [91]:
fig = go.Figure()

groupby_label = new_data.groupby('poet')['poet'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of poets within poems [NEW DATA]',
    xaxis_title_text='Label',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [213]:
new_data.head(5)

,poem,poet,poem_len_by_words
0,برانگیخت لشکر پی قهرشان,6,4.0
1,اسیر عشق تو از هر دو عالم آزادست,5,8.0
2,در کیسه خود بین که تا چه داری,9,8.0
3,غیرت و دین شهامت و مردی,2,6.0
4,چنین بر زمین ریخت خم می ز خویش,3,8.0


##train_test_val

In [94]:
train, test = train_test_split(new_data, test_size=0.2, random_state=1, stratify=new_data['poet'])
train, valid = train_test_split(train, test_size=0.2, random_state=1, stratify=train['poet'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['poem'].values.tolist(), train['poet'].values.tolist()
x_valid, y_valid = valid['poem'].values.tolist(), valid['poet'].values.tolist()
x_test, y_test = test['poem'].values.tolist(), test['poet'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

(60352, 3)
(15088, 3)
(18860, 3)


#Section B:
##task a

In [95]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

### Configuration

In [96]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [97]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 3
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/bert-fa-base-uncased-sentiment-taaghceh/pytorch_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [98]:
# create a key finder based on label 2 id and id to label

label2id = {label: i for i, label in enumerate(poets)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'khajoo': 0, 'asad': 1, 'bahar': 2, 'feyz': 3, 'asadi': 4, 'hafez': 5, 'jami': 6, 'kamal': 7, 'moulavi': 8, 'parvin': 9}
id2label: {0: 'khajoo', 1: 'asad', 2: 'bahar', 3: 'feyz', 4: 'asadi', 5: 'hafez', 6: 'jami', 7: 'kamal', 8: 'moulavi', 9: 'parvin'}


In [ ]:
# setup the tokenizer and configuration

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

### Input Embeddings

In [102]:
idx = np.random.randint(0, len(train))
sample_poem = train.iloc[idx]['poem']
sample_poet = train.iloc[idx]['poet']

print(f'Sample: \n{sample_poem}\n{sample_poet}')

Sample: 
دیده مرغ صراحی بقدح باز کنیم
0


In [105]:
tokens = tokenizer.tokenize(sample_poem)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'  poem: {sample_poem}')
print(f'  Coded_tokens: {tokens}')
print(f'   Tokens: {tokenizer.convert_tokens_to_string(tokens)}')
print(f'Token IDs: {token_ids}')

  poem: دیده مرغ صراحی بقدح باز کنیم
  Coded_tokens: ['دیده', 'مرغ', 'صراحی', 'بق', '##د', '##ح', 'باز', 'کنیم']
   Tokens: دیده مرغ صراحی بقدح باز کنیم
Token IDs: [4275, 6004, 96985, 8997, 2013, 2051, 3012, 3592]


In [106]:
encoding = tokenizer.encode_plus(
    sample_poem,
    max_length=32,
    truncation=True,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=True,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt',  # Return PyTorch tensors
)

print(f'Keys: {encoding.keys()}\n')
for k in encoding.keys():
    print(f'{k}:\n{encoding[k]}')

Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

input_ids:
tensor([[    2,  4275,  6004, 96985,  8997,  2013,  2051,  3012,  3592,     4,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])
token_type_ids:
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])


### Dataset

In [127]:
class PoemsDataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, poems, targets=None, label_list=None, max_len=128):
        self.poems = poems
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len

        
        self.label_map = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}
    
    def __len__(self):
        return len(self.poems)

    def __getitem__(self, item):
        poem = str(self.poems[item])

        if self.has_target:
            target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
            poem,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')
        
        inputs = {
            'poem': poem,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)
        
        return inputs


def create_data_loader(x, y, tokenizer, max_len, batch_size, label_list):
    dataset = PoemsDataset(
        poems=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len, 
        label_list=label_list)
    
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [128]:
label_list = poets
train_data_loader = create_data_loader(train['poem'].to_numpy(), train['poet'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
valid_data_loader = create_data_loader(valid['poem'].to_numpy(), valid['poet'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
test_data_loader = create_data_loader(test['poem'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

In [153]:
sample_data = next(iter(train_data_loader))

print(sample_data.keys())

print(sample_data['poem'])
print(sample_data['input_ids'].shape)
print(sample_data['input_ids'][0, :])
print(sample_data['attention_mask'].shape)
print(sample_data['attention_mask'][0, :])
print(sample_data['token_type_ids'].shape)
print(sample_data['token_type_ids'][0, :])
print(sample_data['targets'].shape)
print(sample_data['targets'])

dict_keys(['poem', 'input_ids', 'attention_mask', 'token_type_ids', 'targets'])
['نهادی دو سه پیل زی شاه پی', 'نه محض جوهر روحی که روح جوهر جانی', 'ما مردانیم شسته بر تنگ دره', 'رانی ام چونان که راند بنده ای  را پادشاهی', 'ساحران در عهد فرعون لعین', 'وزو به عالم جان تحفه گونه گون آید', 'ساقی به دست باش که غم در کمین ماست', 'بسی گوهر ز بام آویختندم', 'الا یا راهب الدیر فهل مرت بک النوق', 'دریاست دهر کشتی خویش استوار دار', 'ما یا آنیم و این دگر فرع', 'بجانب تو روان بود جانم از شوقت', 'وان جامه و رخت تو سخت نغز است', 'چنین گفت کای بخت پیشت رهی', 'چو رامین شد به بند مهر بسته', 'تا کشی گوهری از مخزن غیب']
torch.Size([16, 128])
tensor([    2, 13460,  2897,  3140, 14655,  3029,  3603,  3063,     4,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,  

In [131]:
sample_test = next(iter(test_data_loader))
print(sample_test.keys())

dict_keys(['poem', 'input_ids', 'attention_mask', 'token_type_ids'])


In [155]:
class ClassifierModel(nn.Module):

    def __init__(self, config):
        super(ClassifierModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, 10)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        pooled_output = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids)['pooler_output']
        
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits 

In [143]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

In [170]:
pt_model = ClassifierModel(config=config)
pt_model = pt_model.to(device)

print('pt_model', type(pt_model))

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


pt_model <class '__main__.ClassifierModel'>


In [171]:
for name,param in pt_model.named_parameters():
  if'classifier' not in name:
    param.requires_grad = False
  else:
    print(name)

classifier.weight
classifier.bias


### Training

In [173]:
def simple_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = simple_accuracy(y_true, y_pred)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

def y_loss(y_true, y_pred, losses):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    y = [y_true, y_pred]
    loss = np.mean(losses)

    return y, loss


def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):
            
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)

            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            # calculate the batch loss
            loss = loss_fn(outputs, targets)

            # accumulate all the losses
            losses.append(loss.item())

            y_pred.extend(preds)
            y_true.extend(targets)
    
    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss


def train_op(model, 
             data_loader, 
             loss_fn, 
             optimizer, 
             scheduler, 
             step=0, 
             print_every_step=100, 
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None, 
             clip=0.0):
    
    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1

        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']

        # move tensors to GPU if CUDA is available
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)

        # clear the gradients of all optimized variables
        optimizer.zero_grad()

        # compute predicted outputs by passing inputs to the model
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        
        # convert output probabilities to predicted class
        _, preds = torch.max(outputs, dim=1)

        # calculate the batch loss
        loss = loss_fn(outputs, targets)

        # accumulate all the losses
        losses.append(loss.item())

        # compute gradient of the loss with respect to model parameters
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        # perform optimization step
        optimizer.step()

        # perform scheduler step
        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)

        if eval:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)

    return train_y, train_loss, step, eval_loss_min

def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)
        
        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))
            
            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss
        
        return eval_loss_min


    return eval_cb

In [177]:
from torch import optim

optimizer = optim.Adam(pt_model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model, 
        data_loader=train_data_loader, 
        loss_fn=loss_fn, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        step=step, 
        print_every_step=EEVERY_EPOCH, 
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader, 
        clip=CLIP)
    
    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
    
    eval_y, eval_loss = eval_op(
        model=pt_model, 
        data_loader=valid_data_loader, 
        loss_fn=loss_fn)
    
    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
    
    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

Epochs... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/3772 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

Epoch: 1/3...Step: 1000...Train Loss: 2.304766...Train Acc: 0.116...Valid Loss: 2.284853...Valid Acc: 0.136...
Validation loss decreased (inf --> 2.284853).  Saving model ...


Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

...

Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/943 [00:00<?, ?it/s]

Epoch: 2/3...Step: 4000...Train Loss: 2.256819...Train Acc: 0.159...Valid Loss: 2.244747...Valid Acc: 0.181...
Validation loss decreased (2.254539 --> 2.244747).  Saving model ...
Epoch: 2/3...Step: 4000...Train Loss: 2.256819...Train Acc: 0.159...Valid Loss: 2.244747...Valid Acc: 0.181...
Validation loss decreased (2.254539 --> 2.244747).  Saving model ...


KeyboardInterrupt: ignored

KeyboardInterrupt: ignored

### Prediction

In [178]:
def predict(model, comments, tokenizer, max_len=128, batch_size=32):
    data_loader = create_data_loader(comments, None, tokenizer, max_len, batch_size, None)
    
    predictions = []
    prediction_probs = []

    
    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            
            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs

In [196]:
test_poems = test['poem'].to_numpy()
preds, probs = predict(pt_model, test_poems, tokenizer, max_len=128)

print(preds.shape, probs.shape)

  0%|          | 0/590 [00:00<?, ?it/s]

(18860,) (18860, 10)


In [197]:
preds

array([9, 5, 5, ..., 3, 4, 6])

In [199]:
preds_label = [id2label[t] for t in preds]
preds_label[0:5]

['parvin', 'hafez', 'hafez', 'asad', 'asadi']

##F1 Score and report

In [200]:
y_test, y_pred = test['poet'].values, preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print("--------------classification_report---------------")
print(classification_report(y_test, y_pred, target_names=label_list))

F1: 0.18183054145230343
--------------classification_report---------------
              precision    recall  f1-score   support

      khajoo       0.19      0.06      0.09      1886
        asad       0.17      0.23      0.19      1886
       bahar       0.18      0.21      0.19      1886
        feyz       0.20      0.20      0.20      1886
       asadi       0.28      0.41      0.33      1886
       hafez       0.18      0.26      0.21      1886
        jami       0.15      0.17      0.16      1886
       kamal       0.16      0.12      0.14      1886
     moulavi       0.22      0.09      0.13      1886
      parvin       0.18      0.17      0.17      1886

    accuracy                           0.19     18860
   macro avg       0.19      0.19      0.18     18860
weighted avg       0.19      0.19      0.18     18860



In [223]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

[[108 230 135 240 211 368 175 178 102 139]
 [ 42 427 178 172 363 210 164 122  41 167]
 [ 28 186 396 131 281 199 247 122  90 206]
 [ 61 231 157 370  91 361 230 155  83 147]
 [ 23 258 233  63 769  90 154 103  29 164]
 [ 79 231 181 217 104 495 202 149  80 148]
 [ 52 200 255 146 294 210 317 132  70 210]
 [ 74 261 182 222 209 282 201 229  86 140]
 [ 34 281 270 161 189 241 237 149 178 146]
 [ 53 242 263 138 199 292 202 131  48 318]]


#Section C:
##fine-tuning the model:

##Adam

In [228]:
pt_ft_model = ClassifierModel(config=config)
pt_ft_model = pt_ft_model.to(device)

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


###Training

In [229]:
optimizer = optim.Adam(pt_ft_model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_ft_model, 
        data_loader=train_data_loader, 
        loss_fn=loss_fn, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        step=step, 
        print_every_step=EEVERY_EPOCH, 
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader, 
        clip=CLIP)
    
    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
    
    eval_y, eval_loss = eval_op(
        model=pt_ft_model, 
        data_loader=valid_data_loader, 
        loss_fn=loss_fn)
    
    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
    
    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

Epochs... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/3772 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

###Prediction

In [230]:
test_poems = test['poem'].to_numpy()
preds, probs = predict(pt_ft_model, test_poems, tokenizer, max_len=128)

print(preds.shape, probs.shape)

  0%|          | 0/590 [00:00<?, ?it/s]

(18860,) (18860, 10)


In [231]:
preds

array([9, 6, 5, ..., 0, 8, 9])

In [232]:
preds_label = [id2label[t] for t in preds]
preds_label[0:5]

['parvin', 'jami', 'hafez', 'feyz', 'moulavi']

###F1 Score and report



In [233]:
y_test, y_pred = test['poet'].values, preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print("--------------classification_report---------------")
print(classification_report(y_test, y_pred, target_names=label_list))

F1: 0.31788572635224277
--------------classification_report---------------
              precision    recall  f1-score   support

      khajoo       0.27      0.57      0.36      1886
        asad       0.36      0.50      0.42      1886
       bahar       0.39      0.09      0.14      1886
        feyz       0.45      0.18      0.25      1886
       asadi       0.60      0.36      0.45      1886
       hafez       0.37      0.48      0.42      1886
        jami       0.40      0.21      0.27      1886
       kamal       0.31      0.15      0.21      1886
     moulavi       0.29      0.34      0.32      1886
      parvin       0.27      0.47      0.34      1886

    accuracy                           0.34     18860
   macro avg       0.37      0.34      0.32     18860
weighted avg       0.37      0.34      0.32     18860



In [234]:
print(confusion_matrix(y_test,y_pred))

[[1075   79   12   72    8  338   22   83   55  142]
 [ 138  951   25   10  182   21   65   37  180  277]
 [ 255  237  164   28   80  277  128   88  269  360]
 [ 568  119   11  335    5  362   50   53  149  234]
 [  48  462   44    2  681   17   90   58  197  287]
 [ 573   70   29   66    2  908   33   33   34  138]
 [ 132  302   54   43  102   73  387  106  362  325]
 [ 668  130   26   82   23  146   60  289  180  282]
 [ 293  130   25   84   18  150   79  109  648  350]
 [ 272  168   29   30   35  188   64   66  153  881]]


##SGD

###Training

In [235]:
pt_ft_sgd_model = ClassifierModel(config=config)
pt_ft_sgd_model = pt_ft_sgd_model.to(device)

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [236]:
optimizer = optim.SGD(pt_ft_sgd_model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_ft_sgd_model, 
        data_loader=train_data_loader, 
        loss_fn=loss_fn, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        step=step, 
        print_every_step=EEVERY_EPOCH, 
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader, 
        clip=CLIP)
    
    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
    
    eval_y, eval_loss = eval_op(
        model=pt_ft_sgd_model, 
        data_loader=valid_data_loader, 
        loss_fn=loss_fn)
    
    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
    
    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

Epochs... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/3772 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

###Prediction

In [237]:
test_poems = test['poem'].to_numpy()
preds, probs = predict(pt_ft_sgd_model, test_poems, tokenizer, max_len=128)

print(preds.shape, probs.shape)

  0%|          | 0/590 [00:00<?, ?it/s]

(18860,) (18860, 10)


In [238]:
preds

array([7, 2, 2, ..., 7, 6, 2])

In [239]:
preds_label = [id2label[t] for t in preds]
preds_label[0:5]

['kamal', 'bahar', 'bahar', 'bahar', 'kamal']

###F1 Score and report



In [240]:
y_test, y_pred = test['poet'].values, preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print("--------------classification_report---------------")
print(classification_report(y_test, y_pred, target_names=label_list))

F1: 0.049252678805717806
--------------classification_report---------------
              precision    recall  f1-score   support

      khajoo       0.00      0.00      0.00      1886
        asad       0.00      0.00      0.00      1886
       bahar       0.10      0.36      0.16      1886
        feyz       0.05      0.00      0.00      1886
       asadi       0.00      0.00      0.00      1886
       hafez       0.00      0.00      0.00      1886
        jami       0.10      0.40      0.16      1886
       kamal       0.09      0.19      0.12      1886
     moulavi       0.00      0.00      0.00      1886
      parvin       0.09      0.03      0.05      1886

    accuracy                           0.10     18860
   macro avg       0.04      0.10      0.05     18860
weighted avg       0.04      0.10      0.05     18860



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [241]:
print(confusion_matrix(y_test,y_pred))

[[  0   0 718   4   1   0 751 350   0  62]
 [  0   0 739   3   0   0 818 262   0  64]
 [  1   0 682   4   0   0 628 511   0  60]
 [  0   0 687   1   0   0 749 414   0  35]
 [  2   1 588   3   0   0 776 370   0 146]
 [  1   0 816   0   0   0 684 344   0  41]
 [  2   0 608   2   0   0 755 443   0  76]
 [  0   1 666   2   1   0 794 358   1  63]
 [  1   1 646   1   0   0 690 488   0  59]
 [  3   1 656   1   0   0 718 444   0  63]]


##Result:
####With almost the same amount of training time the Adam optimized model has done a much better job, It seems like that the sgd couldn't handle the difficulty differences of texts(Although it is hard to judge with this state of training). Also, fine-tuned models seem to be able to do much better jobs than previously checked model. 

#Source:
##Code was extracted and used from : https://colab.research.google.com/github/hooshvare/parsbert/blob/master/notebooks/Taaghche_Sentiment_Analysis.ipynb, just a little bit change was needed during the process.
